# 1. Introducció
En aquest notebook es tractarà el cas de Kaggle: **[Melbourne Housing Market](https://www.kaggle.com/anthonypino/melbourne-housing-market)**

El dataset de Kaggle en el què es centra aquest treball conté dades dels habitatges venuts a la ciutat de Melbourne, Austràlia, entre els anys 2016 i 2018. 
L objectiu del treball amb aquest dataset són dos. Per una banda, aprendre a gestionar i resoldre les mostres incompletes per poder aprofitar la major quantitat de mostres possibles. 
La segona, crear models regressors que permetin predir  el preus  de venda per als habitatges que es posaran al mercat en el futur a la ciutat de Melbourne.


In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import model_selection
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import statistics
from matplotlib.legend_handler import HandlerLine2D
from joblib import dump, load
from sklearn.model_selection import RandomizedSearchCV
from random import randint

# 2.Preprocessat dades
Per tal de poder utilitzar les dades del dataset per a entrenar models cal abans assegurar que són adequades per la tasca. En un primer moment cal conèixer el dataset.

In [9]:
def load_dataset(path):
    return pd.read_csv(path, header=0, delimiter=",")
dataset = load_dataset("/home/alexandre/Desktop/APC/Cas Kaggle/APC_Kaggle21/data/Melbourne_housing_FULL.csv")
data = dataset.values

x_housing= data[:, :-1]
y_housing = data[:, -1]
print("Dimensionalitat de la BBDD_housing:", dataset.shape)
print(dataset.head())

Dimensionalitat de la BBDD_housing: (34857, 21)
       Suburb             Address  Rooms Type      Price Method SellerG  \
0  Abbotsford       68 Studley St      2    h        NaN     SS  Jellis   
1  Abbotsford        85 Turner St      2    h  1480000.0      S  Biggin   
2  Abbotsford     25 Bloomburg St      2    h  1035000.0      S  Biggin   
3  Abbotsford  18/659 Victoria St      3    u        NaN     VB  Rounds   
4  Abbotsford        5 Charles St      3    h  1465000.0     SP  Biggin   

        Date  Distance  Postcode  ...  Bathroom  Car  Landsize  BuildingArea  \
0  3/09/2016       2.5    3067.0  ...       1.0  1.0     126.0           NaN   
1  3/12/2016       2.5    3067.0  ...       1.0  1.0     202.0           NaN   
2  4/02/2016       2.5    3067.0  ...       1.0  0.0     156.0          79.0   
3  4/02/2016       2.5    3067.0  ...       2.0  1.0       0.0           NaN   
4  4/03/2017       2.5    3067.0  ...       2.0  0.0     134.0         150.0   

   YearBuilt        

### Tipus d'atributs:
- 1.Suburb: suburbi
- 2.Address: Addreça
- 3.Rooms: Num. d'habitacions
- 4.Price: Preu en Australian dollars
- 5.Method: mètode de venda
- 6.Type:tipus immoble
- 7.SellerG: Real Estate Agent
- 8.Date: data de venda
- 9.Distance: distància al centre in Kilometres
- 10.Regionname: General Region (
- 11.Propertycount: nombre d'immobles al suburbi
- 12.Bedroom2 : num. dormitoris
- 13.Bathroom: num. banys
- 14.Car: num places cotxe
- 15.Landsize: superficie terreny en Metres
- 16.BuildingArea: superficie edificable Metres
- 17.YearBuilt: any construcció
- 18.CouncilArea: concell de govern de la zona
- 19.Lattitude
- 20 Longtitude

Analitzem la l'existència de valors null en les dades:

In [10]:
print(dataset.isnull().sum())

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
dtype: int64


Com es pot observar les mostres presenten una gran heterogeneïtat pel què fa a les característiques que tenen registrades (sense valors null). En alguns atributs com BuildingArea o YearBuilt, 66% de les 
mostres tenen Nan en aquestes caracterísitques. Per la resta d'atributs també hi ha una gran presencia de valors null. Només  un 25% de les mostres contenen tota la informació als atributs. 

Cal aplicar un tractament profund de les dades per substituir els Nan i o eliminar files.